In [1]:
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Parameters
batch_size = 16
img_height = 224
img_width = 224
data_dir = '/content/drive/MyDrive/prenatal'

Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19, ResNet50, DenseNet121, MobileNetV2
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load and prepare dataset (as previously done)
batch_size = 16
img_height = 224
img_width = 224
data_dir = '/content/drive/MyDrive/prenatal'

dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=123
)
class_names = dataset.class_names
print(f"Class names: {class_names}")

# Split dataset into train, val, test
train_size = 0.8
val_size = 0.1
test_size = 0.1

total_count = len(dataset)
train_count = int(total_count * train_size)
val_count = int(total_count * val_size)

train_ds = dataset.take(train_count)
val_ds = dataset.skip(train_count).take(val_count)
test_ds = dataset.skip(train_count + val_count)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

# Function to create a model
def create_model(base_model, input_shape, num_classes, freeze=True):
    base_model.trainable = not freeze
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create individual models
vgg19_base = VGG19(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
resnet50_base = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')


vgg19_model = create_model(vgg19_base, (img_height, img_width, 3), len(class_names))
resnet50_model = create_model(resnet50_base, (img_height, img_width, 3), len(class_names))

# Train individual models
epochs = 10
vgg19_history = vgg19_model.fit(train_ds, validation_data=val_ds, epochs=epochs)
resnet50_history = resnet50_model.fit(train_ds, validation_data=val_ds, epochs=epochs)


# Evaluate models
vgg19_acc = vgg19_model.evaluate(test_ds)[1]
resnet50_acc = resnet50_model.evaluate(test_ds)[1]

print(f"VGG19 Test Accuracy: {vgg19_acc:.2f}")
print(f"ResNet50 Test Accuracy: {resnet50_acc:.2f}")

# Combine predictions (Ensemble)
def ensemble_predictions(models, test_ds):
    all_preds = []
    all_true = []

    for images, labels in test_ds:
        all_true.extend(labels.numpy())
        model_preds = np.zeros((images.shape[0], len(class_names)))

        # Collect predictions from each model
        for model in models:
            preds = model.predict(images)
            model_preds += preds  # Sum the probabilities

        # Average the predictions (for soft-voting)
        avg_preds = model_preds / len(models)
        all_preds.extend(np.argmax(avg_preds, axis=1))

    return np.array(all_true), np.array(all_preds)

# Ensemble models
models = [vgg19_model, resnet50_model, ]
y_true, y_pred = ensemble_predictions(models, test_ds)

# Classification report
print("\nEnsemble Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

Found 1684 files belonging to 2 classes.
Class names: ['Non-standard', 'Standard']
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 254s 3s/step - accuracy: 0.6971 - loss: 1.4835 - val_accuracy: 0.8562 - val_loss: 0.3676
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 55s 660ms/step - accuracy: 0.7998 - loss: 0.4912 - val_accuracy: 0.8062 - val_loss: 0.4773
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 51s 604ms/step - accuracy: 0.8189 - loss: 0.4132 - val_accuracy: 0.8375 - val_loss: 0.3703
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 80s 585ms/step - accuracy: 0.8392 - loss: 0.3733 - val_accuracy: 0.8500 - val_loss: 0.3797
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 53s 635ms/step - accuracy: 0.8475 - loss: 0.3551 - val_accuracy: 0.8250 - val_loss: 0.4329
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 48s 568ms/step - accuracy: 0.8690 - loss: 0.3388 - val_accuracy: 0.8375 - val_loss: 0.3411
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 53s 6